In [2]:
import os
import pandas as pd
import numpy as np
import pprint

# =============================================================================
# BLOCO 1: CONFIGURAÇÃO, CARREGAMENTO, DIVISÃO (80/20) E SEGMENTAÇÃO (CORRIGIDO)
# =============================================================================

# --- 1. CONFIGURAÇÕES GERAIS ---
caminho_raiz = r'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset' # IMPORTANTE: Verifique se este caminho está correto
params_drive_end = {'n': 9, 'd': 0.3126, 'D': 1.537, 'phi_graus': 0.0}
TAXA_AMOSTRAL = 12000

# Dicionários de mapeamento
mapa_tipo_falha = {'IR': 'Pista Interna', 'B': 'Esfera', 'OR': 'Pista Externa', 'Normal': 'Normal'}
mapa_diametro_falha = {'7': '0.007"', '14': '0.014"', '21': '0.021"'}

# --- PARÂMETROS DE SEGMENTAÇÃO ---
tamanho_segmento = 4096
sobreposicao_percentual = 0.3
passo = int(tamanho_segmento * (1 - sobreposicao_percentual))

# --- 2. CARREGAMENTO, DIVISÃO E PROCESSAMENTO ---
dicionario_treino = {}
dicionario_teste = {} # Dicionário para guardar os 20% dos dados normais para teste

print(f"Iniciando a leitura e segmentação dos arquivos em '{caminho_raiz}'...")
print("Dados normais serão divididos em 80% para treino e 20% para teste ANTES da segmentação.")

# Função auxiliar para segmentar um sinal e adicionar ao dicionário
def segmentar_e_adicionar(sinal, metadados, dicionario_alvo, chave_base):
    # Verifica se o sinal é longo o suficiente para pelo menos um segmento
    if len(sinal) < tamanho_segmento:
        # print(f"Aviso: Sinal da base '{chave_base}' muito curto ({len(sinal)} amostras) para gerar segmentos. Ignorando.")
        return 0

    num_segmentos_criados = 0
    for i, inicio in enumerate(range(0, len(sinal) - tamanho_segmento + 1, passo)):
        segmento = sinal[inicio : inicio + tamanho_segmento]
        df_segmento = pd.DataFrame({'amplitude': segmento})

        # Adiciona metadados
        df_segmento['arquivo_origem'] = metadados['nome_arquivo']
        df_segmento['rotacao_rpm'] = metadados['rpm']
        df_segmento['tipo_falha'] = metadados['tipo_falha']
        df_segmento['diametro_falha'] = metadados['diametro_falha']
        df_segmento['local_sensor'] = 'Drive End'

        chave_segmento = f"{chave_base}_seg_{i}"
        dicionario_alvo[chave_segmento] = df_segmento
        num_segmentos_criados += 1
    return num_segmentos_criados

# Loop principal pelos arquivos
for pasta_atual, _, arquivos in os.walk(caminho_raiz):
    for nome_arquivo in arquivos:
        # Processar apenas arquivos .npz
        if nome_arquivo.endswith('.npz'):
            caminho_completo = os.path.join(pasta_atual, nome_arquivo)

            # Decodificação de metadados
            nome_sem_ext = nome_arquivo.replace('.npz', '')
            partes = nome_sem_ext.split('_')
            rpm_str = partes[0]
            is_normal = 'Normal' in nome_arquivo

            metadados = {
                'nome_arquivo': nome_arquivo,
                'rpm': int(rpm_str) if rpm_str.isdigit() else 0,
                'tipo_falha': 'Normal' if is_normal else mapa_tipo_falha.get(partes[1].split('@')[0], 'Desconhecido'),
                'diametro_falha': 'N/A' if is_normal else mapa_diametro_falha.get(partes[2], 'Desconhecido')
            }

            try:
                dados_npz = np.load(caminho_completo)
                sensor_cod = 'DE' # Foco apenas no Drive End, como no seu código original

                if sensor_cod in dados_npz.files:
                    sinal_completo = dados_npz[sensor_cod].ravel()

                    if is_normal:
                        # DIVIDE O SINAL NORMAL EM 80/20
                        ponto_corte = int(len(sinal_completo) * 0.8)
                        sinal_treino = sinal_completo[:ponto_corte]
                        sinal_teste = sinal_completo[ponto_corte:]

                        chave_base_normal = f"{nome_sem_ext}_{sensor_cod}"
                        segmentar_e_adicionar(sinal_treino, metadados, dicionario_treino, f"{chave_base_normal}_treino")
                        segmentar_e_adicionar(sinal_teste, metadados, dicionario_teste, f"{chave_base_normal}_teste")

                    else:
                        # Sinais com falha vão inteiramente para o TREINO
                        # Lógica de chave para arquivos de falha (igual ao seu original)
                        partes_chave = nome_sem_ext.split('_')
                        partes_chave[-1] = partes_chave[-1].rstrip('0123456789')
                        chave_base_falha = "_".join(partes_chave)
                        segmentar_e_adicionar(sinal_completo, metadados, dicionario_treino, chave_base_falha)

            except Exception as e:
                print(f"Erro ao processar o arquivo {nome_arquivo}: {e}")

# --- Relatório Final ---
print("\n--- Processo Concluído! ---")
print(f"Total de segmentos de TREINO (falhas + 80% normais): {len(dicionario_treino)}")
print(f"Total de segmentos de TESTE (20% normais): {len(dicionario_teste)}")

if not dicionario_teste:
    print("\nAVISO: O dicionário de teste está vazio. Verifique se os arquivos 'Normal' existem e se os sinais são longos o suficiente (mínimo de ~21000 amostras).")

if dicionario_treino:
    chave_exemplo_treino = list(dicionario_treino.keys())[0]
    print(f"\nExemplo de um segmento de TREINO (chave: '{chave_exemplo_treino}'):")
    print(dicionario_treino[chave_exemplo_treino].head())

if dicionario_teste:
    chave_exemplo_teste = list(dicionario_teste.keys())[0]
    print(f"\nExemplo de um segmento de TESTE (chave: '{chave_exemplo_teste}'):")
    print(dicionario_teste[chave_exemplo_teste].head())

Iniciando a leitura e segmentação dos arquivos em 'C:\Users\vinic\OneDrive\Documentos\Graduação\TG\Dataset'...
Dados normais serão divididos em 80% para treino e 20% para teste ANTES da segmentação.

--- Processo Concluído! ---
Total de segmentos de TREINO (falhas + 80% normais): 3252
Total de segmentos de TESTE (20% normais): 115

Exemplo de um segmento de TREINO (chave: '1730_B_14_DE_seg_0'):
   amplitude      arquivo_origem  rotacao_rpm tipo_falha diametro_falha  \
0   0.105420  1730_B_14_DE12.npz         1730     Esfera         0.014"   
1  -0.107370  1730_B_14_DE12.npz         1730     Esfera         0.014"   
2  -0.163410  1730_B_14_DE12.npz         1730     Esfera         0.014"   
3   0.118903  1730_B_14_DE12.npz         1730     Esfera         0.014"   
4   0.184039  1730_B_14_DE12.npz         1730     Esfera         0.014"   

  local_sensor  
0    Drive End  
1    Drive End  
2    Drive End  
3    Drive End  
4    Drive End  

Exemplo de um segmento de TESTE (chave: '1730_No

In [3]:
# =============================================================================
# BLOCO DE GERAÇÃO DE DADOS SINTÉTICOS (INCLUINDO SINAIS NORMAIS)
# =============================================================================

# --- 1. FUNÇÃO PARA CÁLCULO DE FREQUÊNCIA TEÓRICA ---
def calcular_frequencias_rolamento(n, fr, d, D, phi_graus=0.0):
    """Calcula as frequências teóricas de falha de um rolamento."""
    phi_rad = np.deg2rad(phi_graus)
    termo_comum = (d / D) * np.cos(phi_rad)
    return {
        'Pista Externa': (n * fr / 2) * (1 - termo_comum),
        'Pista Interna': (n * fr / 2) * (1 + termo_comum),
        'Esfera': (D * fr / (2 * d)) * (1 - termo_comum**2)
    }

# --- 2. PARÂMETROS DE GERAÇÃO (APENAS DRIVE END) ---
amplitudes_referencia = {
    'Drive End': {'Esfera': 0.0001, 'Pista Interna': 0.001, 'Pista Externa': 0.0001}
}
multiplicadores = [0.1, 0.25, 0.5, 0.75, 1.0, 1.5, 2.0, 5.0, 10.0, 25.0, 50.0, 100.0]
fases_para_adicionar_rad = [
    0, np.pi/4, np.pi/2, 3*np.pi/4,
    np.pi, 5*np.pi/4, 3*np.pi/2, 7*np.pi/4
]

# --- 3. IDENTIFICAÇÃO DOS SEGMENTOS NORMAIS DE TREINO ---
# A partir de agora, usamos apenas os dados de treino (dicionario_treino)
segmentos_normais = {
    chave: df for chave, df in dicionario_treino.items()
    if df['tipo_falha'].iloc[0] == 'Normal'
}
print(f"Usando {len(segmentos_normais)} segmentos normais de TREINO do Drive End para gerar dados sintéticos.")

# --- 4. GERAÇÃO E COMBINAÇÃO DOS SINAIS ---
lista_dados_finais = []

# Loop para gerar os sinais com falha sintética
for chave_normal, df_normal in segmentos_normais.items():
    sinal_normal_base = df_normal['amplitude'].values
    rpm_atual = df_normal['rotacao_rpm'].iloc[0]
    
    N_PONTOS = len(sinal_normal_base)
    duracao_s = N_PONTOS / TAXA_AMOSTRAL
    t = np.linspace(0.0, duracao_s, N_PONTOS, endpoint=False)
    
    fr_hz = rpm_atual / 60
    freqs_teoricas = calcular_frequencias_rolamento(fr=fr_hz, **params_drive_end)
    
    for tipo_falha_sintetica in ['Pista Externa', 'Pista Interna', 'Esfera']:
        freq_teorica = freqs_teoricas[tipo_falha_sintetica]
        amp_ref = amplitudes_referencia['Drive End'][tipo_falha_sintetica]
            
        for mult in multiplicadores:
            for fase in fases_para_adicionar_rad:
                amplitude_final = amp_ref * mult
                sinal_falha_sintetico = amplitude_final * np.sin(2 * np.pi * freq_teorica * t + fase)
                sinal_final_combinado = sinal_normal_base + sinal_falha_sintetico
                
                lista_dados_finais.append({
                    'sinal_final': sinal_final_combinado,
                    'tipo_falha_adicionada': tipo_falha_sintetica,
                    'rpm': rpm_atual,
                    'multiplicador_amplitude': mult,
                    'fase_adicionada_rad': fase,
                    'base_normal': chave_normal,
                    'local_sensor': 'Drive End'
                })

# --- 5. ADIÇÃO DOS SEGMENTOS NORMAIS ORIGINAIS ---
print(f"\nAdicionando os {len(segmentos_normais)} segmentos normais ao conjunto de dados final...")
for chave_normal, df_normal in segmentos_normais.items():
    lista_dados_finais.append({
        'sinal_final': df_normal['amplitude'].values,
        'tipo_falha_adicionada': 'Normal', # Classe "Normal"
        'rpm': df_normal['rotacao_rpm'].iloc[0],
        'multiplicador_amplitude': 0, # Não aplicável
        'fase_adicionada_rad': 0, # Não aplicável
        'base_normal': chave_normal,
        'local_sensor': 'Drive End'
    })

# --- 6. DATAFRAME FINAL COM DADOS SINTÉTICOS E NORMAIS ---
df_final_completo = pd.DataFrame(lista_dados_finais)

print("\n\n--- Geração de Dados Concluída! ---")
print(f"Total de {len(df_final_completo)} segmentos (sintéticos + normais) gerados para o Drive End.")
print("\n--- Exemplo do DataFrame Final Gerado ---")
# Mostra as últimas linhas para vermos os exemplos de sinais normais
print(df_final_completo.drop(columns=['sinal_final']).tail())

Usando 470 segmentos normais de TREINO do Drive End para gerar dados sintéticos.

Adicionando os 470 segmentos normais ao conjunto de dados final...


--- Geração de Dados Concluída! ---
Total de 135830 segmentos (sintéticos + normais) gerados para o Drive End.

--- Exemplo do DataFrame Final Gerado ---
       tipo_falha_adicionada   rpm  multiplicador_amplitude  \
135825                Normal  1797                      0.0   
135826                Normal  1797                      0.0   
135827                Normal  1797                      0.0   
135828                Normal  1797                      0.0   
135829                Normal  1797                      0.0   

        fase_adicionada_rad                   base_normal local_sensor  
135825                  0.0  1797_Normal_DE_treino_seg_62    Drive End  
135826                  0.0  1797_Normal_DE_treino_seg_63    Drive End  
135827                  0.0  1797_Normal_DE_treino_seg_64    Drive End  
135828                  

In [33]:
import numpy as np
import pandas as pd
from scipy.stats import skew

# =============================================================================
# BLOCO DE EXTRAÇÃO DE ATRIBUTOS DOS SINAIS SINTÉTICOS GERADOS
# =============================================================================

# --- 1. FUNÇÕES PARA CÁLCULO DOS ATRIBUTOS ---

# --- DOMÍNIO DO TEMPO ---
def calcular_tf2_std(sinal):
    """Calcula o Desvio Padrão (TF2)."""
    return np.std(sinal)

def calcular_tf3_rms(sinal):
    """Calcula o valor RMS (TF3)."""
    return np.sqrt(np.mean(sinal**2))

def calcular_tf4_fator_forma(sinal):
    """Calcula o Fator de Forma (TF4)."""
    rms = calcular_tf3_rms(sinal)
    media_abs = np.mean(np.abs(sinal))
    return rms / media_abs if media_abs != 0 else 0

# --- DOMÍNIO DA FREQUÊNCIA ---
def calcular_features_frequencia(sinal, taxa_amostral):
    """Calcula FF2, FF3 e FF5 de uma só vez."""
    N = len(sinal)
    if N == 0: return None, None, None

    # Aplica a FFT e obtém o espectro de amplitude
    espectro = np.abs(np.fft.fft(sinal)[0:N//2])
    freqs = np.fft.fftfreq(N, 1 / taxa_amostral)[:N//2]
    
    soma_espectro = np.sum(espectro)
    if soma_espectro == 0: return 0, 0, 0

    # FF2: Frequência Central
    ff2_freq_central = np.sum(freqs * espectro) / soma_espectro
    
    # FF3: RMS da Frequência
    ff3_rms_freq = np.sqrt(np.sum((freqs**2) * espectro) / soma_espectro)
    
    # FF4: Desvio Padrão da Frequência (necessário para FF5)
    ff4_std_freq = np.sqrt(np.sum(((freqs - ff2_freq_central)**2) * espectro) / soma_espectro)
    
    # FF5: Assimetria Espectral
    numerador_ff5 = np.sum(((freqs - ff2_freq_central)**3) * espectro) / soma_espectro
    ff5_assimetria = numerador_ff5 / (ff4_std_freq**3) if ff4_std_freq != 0 else 0
        
    return ff2_freq_central, ff3_rms_freq, ff5_assimetria


# --- 2. PROCESSAMENTO E EXTRAÇÃO DE FEATURES ---

print("--- Iniciando extração de atributos de cada SINAL SINTÉTICO (Drive End) ---")
lista_de_features = []

# Itera sobre o DataFrame com os sinais sintéticos do Drive End
# O método .itertuples() é mais rápido que iterrows()
for linha in df_final_de.itertuples():
    sinal_sintetico = linha.sinal_final
    
    # Calcula os atributos
    tf2 = calcular_tf2_std(sinal_sintetico)
    tf3 = calcular_tf3_rms(sinal_sintetico)
    tf4 = calcular_tf4_fator_forma(sinal_sintetico)
    ff2, ff3, ff5 = calcular_features_frequencia(sinal_sintetico, TAXA_AMOSTRAL)
    
    # Armazena os resultados junto com os metadados
    lista_de_features.append({
        'tipo_falha_adicionada': linha.tipo_falha_adicionada,
        'rpm': linha.rpm,
        'multiplicador_amplitude': linha.multiplicador_amplitude,
        'fase_adicionada_rad': linha.fase_adicionada_rad,
        'base_normal': linha.base_normal,
        'TF2_std': tf2,
        'TF3_rms': tf3,
        'TF4_fator_forma': tf4,
        'FF2_freq_central': ff2,
        'FF3_rms_freq': ff3,
        'FF5_assimetria_espectral': ff5
    })

# --- 3. CRIAÇÃO DO DATAFRAME FINAL DE ATRIBUTOS ---
df_features_sinteticas = pd.DataFrame(lista_de_features)

print(f"\nExtração concluída! Atributos de {len(df_features_sinteticas)} sinais sintéticos foram calculados.")
print("\n--- Exemplo do DataFrame final com os atributos extraídos ---")
print(df_features_sinteticas.head())

# Mostra informações sobre o DataFrame final
print("\n--- Informações do DataFrame de Atributos Sintéticos ---")
df_features_sinteticas.info()

--- Iniciando extração de atributos de cada SINAL SINTÉTICO (Drive End) ---

Extração concluída! Atributos de 169344 sinais sintéticos foram calculados.

--- Exemplo do DataFrame final com os atributos extraídos ---
  tipo_falha_adicionada   rpm  multiplicador_amplitude  fase_adicionada_rad  \
0         Pista Externa  1730                      0.1             0.000000   
1         Pista Externa  1730                      0.1             0.785398   
2         Pista Externa  1730                      0.1             1.570796   
3         Pista Externa  1730                      0.1             2.356194   
4         Pista Externa  1730                      0.1             3.141593   

            base_normal   TF2_std   TF3_rms  TF4_fator_forma  \
0  1730_Normal_FE_seg_0  0.073620  0.080450         1.256159   
1  1730_Normal_FE_seg_0  0.073620  0.080450         1.256160   
2  1730_Normal_FE_seg_0  0.073619  0.080450         1.256159   
3  1730_Normal_FE_seg_0  0.073619  0.080449         1